# CR ML Road Map


- Step 1:
    - TBA
- Step 2:
    - TBA 
- Step 3:
    - TBA

## Import Packages

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
# basic python package
import importlib
import numpy as np
import time
import logging
importlib.reload(logging)
logging.basicConfig(level = logging.INFO)

# python ploting packages
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, BoundaryNorm
from matplotlib.collections import LineCollection
from matplotlib import cm


# self-define classes
from script import CR_ML_Class as CR
from script import load_mock_data as LD


# tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
print("Tensorflow Version is {}".format(tf.__version__))
print("Keras Version is {}".format(tf.keras.__version__))
!nvudia-smi

Tensorflow Version is 2.4.1
Keras Version is 2.4.0
/bin/bash: nvudia-smi: command not found


## Transfrom Mock Data from Text File to Numpy Dataframe 

- Data Description:


    - parameter : propagation and source parameters 
        numpy shape: (# of mock data, 14)
            #parameter: original parameter 
            #new_parameter: recalculate the normal factor and Ap 

            raw1=D_0, 
            raw2=\delta, 
            raw3=z_h, 
            raw4=v_A, 
            raw5=\eta, 
            raw6=A_p, 
            raw7=\nu_1, 
            raw8=\nu_2, 
            raw9=log10(R_{br,1}), 
            raw10=\nu_3, 
            raw11=log10(R_{br,2}), 
            raw12=N_{Li}, 
            raw13=N_{Be}, 
            raw14=N_{O}


    - data: Mock data
        numpy shape: (# of mock data, 84, 6)
            84 means there are 84 energy bins from 1.000e-03 to 1.105e+05
            6 means there are the spectrum for E, Li, Be, B, C, O
            #data_0: original mocak data
            #modify_data_0: reshape data accroding to recalculated parameters


     - chi: chi-square 
        numpy shape: (# of mock data)
            #new_chi: chi-square from "modify_data"

In [74]:
%%time
text_Data_path = "../Data/Text_Mock_data/"

# mockdata_1 = CR.Mock_Data_to_NumpyArray(text_Data_path + "res_3x_1.txt")
mockdata_1 = CR.Mock_Data_to_NumpyArray(text_Data_path + "return_4.txt")
parameter, data, chisq = mockdata_1.parameter, mockdata_1.spectrum, mockdata_1.chisq

logging.info("Data Shape for 'parameter': {}".format(parameter.shape))
logging.info("Data Shape for 'data': {}".format(data.shape))
logging.info("Data Shape for 'chisq': {}".format(chisq.shape))

INFO:root:Thu Aug 05 17:27:49 2021
INFO:root:Now loading...
INFO:root:Total data: 30000
INFO:root:Time consumption : 0.4221 min
INFO:root:Data Shape for 'parameter': (30000, 15)
INFO:root:Data Shape for 'data': (2520000, 11)
INFO:root:Data Shape for 'chisq': (30000,)


CPU times: user 25.2 s, sys: 566 ms, total: 25.7 s
Wall time: 25.7 s


## ReCalculate Normalized Factor for Mock Data

- 1: Using 
    ```
    CR.Mock_Data_Rescale(parameter, parameter, data)
    ```
    to split 'data' into Li, Be, B, C and O `spectra`.  
    Note that we put the same 'parameter' here because we have not get new normalized factor yet
    
    - `spectra`: Mock data
        numpy shape: (# of mock data, 84, 6)
            84 means there are 84 energy bins from 1.000e-03 to 1.105e+05
            6 means there are the spectrum for E, Li, Be, B, C, O
            #data_0: original mocak data
            #modify_data_0: reshape data accroding to recalculated paramete

In [111]:
%%time
spectra_data = CR.Mock_Data_Rescale(origin_parameter=parameter, new_parameter=parameter, spectrum=data, usedata=False)
logging.info("Data Shape for 'spectra_data': {}".format(spectra_data.data.shape))
logging.info("There are {} mock data.".format(spectra_data.data.shape[0]))
logging.info("For each mock data, there are {} energy bins.".format(spectra_data.data.shape[1]))
logging.info("{} corresponding to E, Li, Be, B, C and O.".format(spectra_data.data.shape[2]))

INFO:root:Data Shape for 'spectra_data': (30000, 84, 6)
INFO:root:There are 30000 mock data.
INFO:root:For each mock data, there are 84 energy bins.
INFO:root:6 corresponding to E, Li, Be, B, C and O.


CPU times: user 169 ms, sys: 79.1 ms, total: 248 ms
Wall time: 248 ms


In [93]:
%%time
importlib.reload(CR)
new_Ap = CR.ReCalculateAp(spectra_data.data[:100]) 
new_Ap.GetBestAp()

INFO:root:Thu Aug 05 17:42:08 2021
INFO:root:Finding best Ap
INFO:root:=====START=====
100%|██████████| 100/100 [00:00<00:00, 103.38it/s]
INFO:root: Time Cost for this Step : 0.0162 min
INFO:root:=====Finish=====
INFO:root:Time Cost : 0.0163 min


CPU times: user 983 ms, sys: 8.05 ms, total: 991 ms
Wall time: 986 ms


In [95]:
%%time
importlib.reload(CR)
new_normalized_factor = CR.ReCalculateN(spectra_data.new_parameter[:100],spectra_data.data[:100],ap=new_Ap.ap_5)
new_normalized_factor.GetBestN()

INFO:root:Thu Aug 05 17:42:21 2021
INFO:root:Finding New Normalized Factor
INFO:root:=====START=====
100%|██████████| 100/100 [00:01<00:00, 80.77it/s]
INFO:root: Time Cost for this Step : 0.0207 min
INFO:root:=====Finish=====
INFO:root:Time Cost : 0.0208 min


CPU times: user 1.25 s, sys: 6.05 ms, total: 1.26 s
Wall time: 1.26 s


In [96]:
%%time
new_parameter = CR.New_Parameter(spectra_data.new_parameter[:100],new_normalized_factor.new_factor,ap_5=new_Ap.ap_5).new_parameter

INFO:root:Thu Aug 05 17:42:25 2021
INFO:root:Time Cost : 0.0000 min


CPU times: user 2.91 ms, sys: 1 µs, total: 2.91 ms
Wall time: 2.21 ms


In [99]:
%%time
new_spectra_data = CR.Mock_Data_Rescale(parameter[:100], new_parameter, spectra_data.data[:100],usedata = True)



# para_1_sigma, data_1_sigma =  Select_Sample(mock_0.new_parameter,mock_0.data,chi_0,1).Sample()

CPU times: user 1.84 ms, sys: 27 µs, total: 1.87 ms
Wall time: 1.3 ms


In [118]:
importlib.reload(CR)

help(CR.Mock_Data_Processing.spectrum_ratio)

Help on function spectrum_ratio in module script.CR_ML_Class:

spectrum_ratio(self, mock=True)
    Usage: 
        Mock_Data_Processing(parameter, E, Li, Be, B, C, O, data, usedata = False).spectrum_ratio(mock = True)
    Return:
        Null
    Item:
        self.total_data_in_ratio

